# Code

In [1]:
its_jupyter_notebook = True

In [2]:
import sys
sys.path.insert(0, '..')
import pickle
import seaborn as sns
import pandas as pd
import os
import random
import numpy as np
import time
if its_jupyter_notebook:
    sys.argv = [''] #Remove this if it's not a jupyter notebook!

In [3]:
ROOT_DIR = os.path.dirname(os.path.abspath('.'))
original_files_dir = os.path.join(ROOT_DIR, 'dataset', 'original_files')
processed_files_dir = os.path.join(ROOT_DIR, 'dataset', 'processed_files')
rna_rna_files_dir = os.path.join(ROOT_DIR, 'dataset', 'rna_rna_pairs')
df_or = pd.read_csv(os.path.join(original_files_dir, "rise_paris_tr.new.mapped_interactions.tx_regions.txt"), sep = '\t')

### Import

In [4]:
file_train = os.path.join(rna_rna_files_dir, "gene_pairs_training_random.txt")
file_test =  os.path.join(rna_rna_files_dir, "gene_pairs_test_random.txt")
file_val = os.path.join(rna_rna_files_dir, "gene_pairs_val_random.txt")

assert (os.path.isfile(file_test))&(os.path.isfile(file_val))&(os.path.isfile(file_test))
with open(file_test, "rb") as fp:   # Unpickling
    gene_pairs_test = pickle.load(fp)
with open(file_val, "rb") as fp:   # Unpickling
    gene_pairs_val = pickle.load(fp)
with open(file_train, "rb") as fp:   # Unpickling
    gene_pairs_train = pickle.load(fp)

In [5]:
df = pd.read_csv(os.path.join(processed_files_dir,"df_annotation_files_cleaned.csv"), sep = ',')

/home/gbini/miniconda3/envs/rnarna/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape[0]

171281

In [7]:
len(gene_pairs_val) + len(gene_pairs_test) + len(gene_pairs_train)

167706

In [8]:
excl1 = pd.read_csv(os.path.join(processed_files_dir,"exclude", "rise_ids.abundant_RNAs.txt"), sep = '\t')

In [9]:
excl2 = pd.read_csv(os.path.join(processed_files_dir,"exclude", "rise_tr.exclude_riseids.no_aboundant.txt"), sep = '\t', header = None)

In [10]:
excl3 = pd.read_csv(os.path.join(processed_files_dir,"exclude","swappereccio.exclude_couples.no_aboundant.txt"), sep = '\t', header = None)

In [12]:
set(excl1.name)

{'SINE_5S-Deu-L2',
 'SINE_7SL',
 'SINE_tRNA',
 'SINE_tRNA-RTE',
 'rRNA_unknown',
 'scRNA_unknown',
 'snRNA_unknown',
 'srpRNA_unknown',
 'tRNA_unknown'}

### Let's start with excl3

Non so se sono ordinate nel modo in cui ho ordinato io le coppie, percio faro cosi: credo due liste, una con gene1_gene2 e l altra con gene2_gene1 e poi faccio l inner di entrambe le liste col df.

In [11]:
excl3 = excl3[0].str.extractall("(.*)-(.*)").reset_index()

In [12]:
excl3['drop1'] = excl3[[0, 1]].agg('_'.join, axis=1)
excl3['drop2'] = excl3[[1, 0]].agg('_'.join, axis=1)

In [13]:
excl3_list = list(excl3['drop1']) + list(excl3['drop2'])

### Let's work with excl1 and excl2

In [14]:
rise_to_exclude = list(excl2[0]) + list(excl1.rise_id)

In [15]:
subset = df_or[~(df_or.rise_id.isin(rise_to_exclude))].reset_index()[['gene_1', 'gene_2']]
subset['drop1'] = subset[['gene_1', 'gene_2']].agg('_'.join, axis=1)
subset['drop2'] = subset[['gene_2', 'gene_1']].agg('_'.join, axis=1)
excl12_list = list(subset['drop1']) + list(subset['drop2'])

### Let's exclude CDS-CDS

In [16]:
subset = df_or[(df_or['tx_id_1_localization'] == 'CDS')&(df_or['tx_id_2_localization'] == 'CDS')][['gene_1', 'gene_2']]
subset['drop1'] = subset[['gene_1', 'gene_2']].agg('_'.join, axis=1)
subset['drop2'] = subset[['gene_2', 'gene_1']].agg('_'.join, axis=1)

In [17]:
cds_cds_list = list(subset['drop1']) + list(subset['drop2'])

### Exclude

In [18]:
exclude = excl3_list + excl12_list + cds_cds_list

exclude = set(exclude) #remove duplicates

In [19]:
gene_pairs_test_filtered = list(set(gene_pairs_test) - exclude)
gene_pairs_val_filtered = list(set(gene_pairs_val) - exclude)
gene_pairs_train_filtered = list(set(gene_pairs_train) - exclude)

In [20]:
def print_perc_01(df):
    true = list(set(df[df.there_is_interaction == True].couples))
    false = list(set(df[df.there_is_interaction == False].couples))
    t = len(true)
    f = len(false)
    return np.round(t/(t+f)*100, 2)

In [21]:
print('Tot dati prima:')
print(len(gene_pairs_test) + len(gene_pairs_val) + len(gene_pairs_train))

print('Tot dati dopo:')
print(len(gene_pairs_test_filtered) + len(gene_pairs_val_filtered) + len(gene_pairs_train_filtered))

print('\n')

print('Test prima:')
print(len(gene_pairs_test))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_test)])}")
print('Val prima:')
print(len(gene_pairs_val))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_val)])}")
print('Train prima:')
print(len(gene_pairs_train))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_train)])}")

print('\n')

print('Test dopo:')
print(len(gene_pairs_test_filtered))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_test_filtered)])}")
print('Val dopo:')
print(len(gene_pairs_val_filtered))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_val_filtered)])}")
print('Train dopo:')
print(len(gene_pairs_train_filtered))
print(f"Perc interaction: {print_perc_01(df[df.couples.isin(gene_pairs_train_filtered)])}")

Tot dati prima:
167706
Tot dati dopo:
95562


Test prima:
34013
Perc interaction: 50.22
Val prima:
17120
Perc interaction: 49.55
Train prima:
116573
Perc interaction: 50.0


Test dopo:
17527
Perc interaction: 5.0
Val dopo:
8853
Perc interaction: 4.83
Train dopo:
69182
Perc interaction: 19.05


### Export unbalanced new lists of train, test, val

In [22]:
new_file_train = os.path.join(rna_rna_files_dir, "gene_pairs_training_random_filtered_unbalanced.txt")
new_file_test =  os.path.join(rna_rna_files_dir, "gene_pairs_test_random_filtered_unbalanced.txt")
new_file_val = os.path.join(rna_rna_files_dir, "gene_pairs_val_random_filtered_unbalanced.txt")

with open(new_file_train, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_train_filtered, fp)

with open(new_file_val, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_val_filtered, fp)
    
with open(new_file_test, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_test_filtered, fp)

In [23]:
gene_pairs_test_filtered_un = gene_pairs_test_filtered.copy()
gene_pairs_train_filtered_un = gene_pairs_train_filtered.copy()
gene_pairs_val_filtered_un = gene_pairs_val_filtered.copy()

### Bilanciamento

I dati sono troppo sbilanciati, provo a bilanciare il set.

In [24]:
test = df[df.couples.isin(gene_pairs_test_filtered)].reset_index(drop = True)
val = df[df.couples.isin(gene_pairs_val_filtered)].reset_index(drop = True)
train = df[df.couples.isin(gene_pairs_train_filtered)].reset_index(drop = True)

In [25]:
def balance(df):
    true = list(set(df[df.there_is_interaction == True].couples))
    false = list(set(df[df.there_is_interaction == False].couples))
    n_true = len(true)
    false_sample = random.sample( false, n_true)
    return true + false_sample

gene_pairs_train_filtered = balance(train)
gene_pairs_val_filtered = balance(val)
gene_pairs_test_filtered = balance(test)

In [26]:
print(len(gene_pairs_train_filtered))
print(len(gene_pairs_val_filtered))
print(len(gene_pairs_test_filtered))

26360
856
1752


In [27]:
def check_intersection(split, split_un):
    split = set(split)
    split_un = set(split_un)
    intersection = split.intersection(split_un)
    return len(split) == len(intersection)
    
assert check_intersection(gene_pairs_train_filtered, gene_pairs_train_filtered_un)
assert check_intersection(gene_pairs_val_filtered, gene_pairs_val_filtered_un)
assert check_intersection(gene_pairs_test_filtered, gene_pairs_test_filtered_un)

### Export balanced new lists of train, test, val

In [28]:
new_file_train = os.path.join(rna_rna_files_dir, "gene_pairs_training_random_filtered.txt")
new_file_test =  os.path.join(rna_rna_files_dir, "gene_pairs_test_random_filtered.txt")
new_file_val = os.path.join(rna_rna_files_dir, "gene_pairs_val_random_filtered.txt")

with open(new_file_train, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_train_filtered, fp)

with open(new_file_val, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_val_filtered, fp)
    
with open(new_file_test, "wb") as fp: #Pickling
    pickle.dump(gene_pairs_test_filtered, fp)